<a href="https://colab.research.google.com/github/jhonnycodedev/data-science/blob/jhonnycodedev/Lista%20NPS%20Positivo_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [2]:
from google.colab import files
uploaded = files.upload()


Saving Lista NPS Positivo_V4.xlsx to Lista NPS Positivo_V4.xlsx


In [4]:
import io

# Substitua 'nome_do_arquivo.xlsx' pelo nome do seu arquivo
df = pd.read_excel(io.BytesIO(uploaded['Lista NPS Positivo_V4.xlsx']), engine='openpyxl')


In [ ]:
# Verificando as primeiras linhas
#print(df.head())

# Resumo estatístico
#print(df.describe())

# Informações sobre o DataFrame
#print(df.info())


In [5]:
# Função para classificar as notas
df['target'] = df['nota'].apply(lambda x: 'Promotor' if x >= 9 else ('Neutro' if x >= 7 else 'Detrator'))


In [8]:
#print(df.groupby('target').count())

In [18]:

# Filtrar para "Grupo de Produto" == "Grupo 2"
grupo_2_df = df[(df['Grupo de Produto'] == 'Grupo 2') & (df['mercado'] == 'BRASIL')]


# Definir as regiões com base nos estados
regioes = {
    "Norte": ['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'],
    "Nordeste": ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
    "Centro-Oeste": ['DF', 'GO', 'MT', 'MS'],
    "Sul": ['PR', 'RS', 'SC'],
    "Sudeste": ['ES', 'MG', 'RJ', 'SP']
}

# Função para definir a região com base no estado
def obter_regiao(estado):
    for regiao, estados in regioes.items():
        if estado in estados:
            return regiao
    return 'Outros(não identificado)'  # Para estados fora das regiões definidas



In [24]:
# Crie a coluna 'regiao' se ainda não tiver feito
grupo_2_df['regiao'] = grupo_2_df['estado'].apply(obter_regiao)

# Crie um dicionário para armazenar os DataFrames
dfs_por_regiao = {}

# Use o groupby para dividir o DataFrame por 'regiao'
for regiao, df_regiao in grupo_2_df.groupby('regiao'):
    dfs_por_regiao[regiao] = df_regiao


<ipython-input-24-40cb2707167c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grupo_2_df['regiao'] = grupo_2_df['estado'].apply(obter_regiao)


In [26]:
df_norte = dfs_por_regiao['Norte']
#df_sul = dfs_por_regiao['Sul']
#df_centro-oeste = dfs_por_regiao['Centro-Oeste']
#df_nordeste = dfs_por_regiao['Nordeste']
#df_sudeste = dfs_por_regiao['Suldeste']


In [27]:
print(df_norte)

                          db-id control_id                 invite_id  \
479    621cdea78c155d08cf3b6aaf     T72969  620a608ab0d07664995be093   
761    6238694fe64df04c22041b1b     CSFINY  620a63695e1e5064865e2188   
861    624ee4305db3924b60c10075     D3TI43  620a608ab0d07664995bdf20   
1696   62850c9eb0f122255b05a0a0     UQYCJ5  627d02b87b9abc208450b9b4   
2534   62a5b9c3e4780323d9ae63ef     VWKLLO  628baebecc34dd7c1edcf1a9   
...                         ...        ...                       ...   
23295  66a0f173a22fbd001aa60a8a   2SG8G4QT  6696c8c81710790013c326e1   
23420  66a3aa7af88b3800351ff407   RV9R9L8Z  6696c8c81710790013c33071   
23507  66a7e76c4e62f300258a4310   AWTB7XGW  6696c87be5dcc70013763583   
24901  66f7ef619dce9500208197d7   7K4SKZ19  6696c8a4e5dcc7001376e847   
24946  66fabd3bdc0da8002517444a   Y4JTRHEH  6696c87be5dcc700137647bf   

      Tipo Produto Grupo de Produto Familia do Produto Modelo do Produto  \
479         Tipo 1          Grupo 2         Familia 20     

In [31]:
# Extrair o ano da data de resposta e manter a coluna 'Periodo de Pesquisa'
df_teste = df_norte['safra'] = pd.to_datetime(grupo_2_df['data_resposta']).dt.year

print(df_teste)

6        2022
26       2022
31       2022
36       2022
56       2022
         ... 
25075    2024
25076    2024
25078    2024
25079    2024
25080    2024
Name: data_resposta, Length: 2638, dtype: int32


<ipython-input-31-e064d7bbaa79>:2: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_teste = df_norte['safra'] = pd.to_datetime(grupo_2_df['data_resposta']).dt.year


In [ ]:
# Agrupar por 'regiao', 'ano_resposta', 'Periodo de Pesquisa' e 'target'
# Para contar as ocorrências de cada avaliação (Promotor, Neutro, Detrator) por ano e período
resultado_por_ano_periodo = grupo_2_df.groupby(['regiao', 'ano_resposta', 'Periodo de Pesquisa', 'target']).size().unstack(fill_value=0)



In [ ]:
# Exibir o resumo resultante
print(resultado_por_ano_periodo)

# (Opcional) Salvar o resultado em um novo arquivo
#resultado_por_ano_periodo.to_excel('Resumo_Impacto_Notas_Grupo_2_Por_Ano_e_Periodo.xlsx')
